## Analysis of Men's and Women's 2023 NCAA DI Swimming Championships

I will explore the results of the 2023 NCAA championship meet and answer a few questions:
1. Which team(s) did best (and how should that be defined)?
2. Are there any differences in performance based on a swimmer's year?
3. What are the average improvements from qualifying times to preliminaries to finals? Do any factors impact this?

### Loading and Inspecting Data

Most of this step was handled in `buildData_2023NCAAs.ipynb` where I converted PDF results to a CSV file. I'll load the CSVs and do some inspection and any small cleaning required.

In [89]:
import pandas as pd
import numpy as np

In [94]:
# Load the men's and women's results and combine dataframes
dfResultsM = pd.read_csv('NCAA_M2023.csv')
dfResultsW = pd.read_csv('NCAA_W2023.csv')
# Division column used to distinguish between DI, DII, DIII and men/women
dfResultsM['Division'] = 'Men\'s DI'
dfResultsW['Division'] = 'Women\'s DI'
# results have same columns by construction, can just be concatenated to combine
dfResults = pd.concat([dfResultsM, dfResultsW], ignore_index=True)
dfResults.head()

,Unnamed: 0,Event,Category,Name1,Year1,Name2,Year2,Name3,Year3,Name4,Year4,School,QualifyingTime,Time,Place,Points,Division
0,0,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,"Stokowski, Kacper",SR,"Hunter, Mason",5Y,"Korstanje, Nyls",SR,"Curtiss, David",SO,NC State,1:22.25,1:20.67,1,40.0,Men's DI
1,1,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,"Dolan, Jack",SR,"Marchand, Leon",SO,"McCusker, Max",5Y,"Kulow, Jonny",FR,Arizona St,1:21.69,1:21.07,2,34.0,Men's DI
2,2,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,"Chaney, Adam",JR,"Savickas, Aleksas",FR,"Friese, Eric",SR,"Liendo, Josh",FR,Florida,1:21.73,1:21.14,3,32.0,Men's DI
3,3,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,"Seeliger, Bjorn",JR,"Bell, Liam",SR,"Rose, Dare",JR,"Alexy, Jack",SO,California,1:22.84,1:21.24,4,30.0,Men's DI
4,4,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,"Burns, Brendan",SR,"Mathias, Van",5Y,"Frankel, Tomer",JR,"Wight, Gavin",JR,Indiana,1:23.52,1:21.52,5,28.0,Men's DI


Everything looks ok after reading the two dataframes in, except as I wrote/read I ended up with 2 index columns, so I can drop the 'Unnamed: 0' column. I'll also modify the name columns to get First + Last names. Finally, inspecting some of the object columns showed leading/trailing whitespaces, so I'll remove those.

In [95]:
dfResults = dfResults.drop(['Unnamed: 0'], axis=1)
name = pd.DataFrame()
for col in ['Name1','Name2','Name3','Name4']:
    name[['Last', 'First']] = dfResults[col].str.split(',',expand=True)
    dfResults[col] = name['First'] + ' ' + name['Last']
dfObj = dfResults.select_dtypes(['object'])
dfResults[dfObj.columns] = dfObj.apply(lambda x: x.str.strip())
dfResults.head()

,Event,Category,Name1,Year1,Name2,Year2,Name3,Year3,Name4,Year4,School,QualifyingTime,Time,Place,Points,Division
0,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,Kacper Stokowski,SR,Mason Hunter,5Y,Nyls Korstanje,SR,David Curtiss,SO,NC State,1:22.25,1:20.67,1,40.0,Men's DI
1,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,Jack Dolan,SR,Leon Marchand,SO,Max McCusker,5Y,Jonny Kulow,FR,Arizona St,1:21.69,1:21.07,2,34.0,Men's DI
2,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,Adam Chaney,JR,Aleksas Savickas,FR,Eric Friese,SR,Josh Liendo,FR,Florida,1:21.73,1:21.14,3,32.0,Men's DI
3,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,Bjorn Seeliger,JR,Liam Bell,SR,Dare Rose,JR,Jack Alexy,SO,California,1:22.84,1:21.24,4,30.0,Men's DI
4,Event 1 Men 200 Yard Medley Relay,Timed Final Relay,Brendan Burns,SR,Van Mathias,5Y,Tomer Frankel,JR,Gavin Wight,JR,Indiana,1:23.52,1:21.52,5,28.0,Men's DI


In [96]:
dfResults.dtypes

Event              object
Category           object
Name1              object
Year1              object
Name2              object
Year2              object
Name3              object
Year3              object
Name4              object
Year4              object
School             object
QualifyingTime     object
Time               object
Place              object
Points            float64
Division           object
dtype: object

Looking at the data types, I see everything is an object other than the points (there are some half point results in the case of a tie, so float is required for those). The Place could be an int, although currently non-placing swimmers are given a place of '---'. This could be made to be NaN so that the type could be changed to int. The times - other than 'DQ' (disqualified) and 'DFS' (declared false start) - could be stored as datetimes to make operations with them easier. Everythig else should stay an object.

Inspecting the dataframe more carefully, I noticed a few relay entries where the swimmer name columns dropped last names so ended up with NaN values after some manipulations. Since there are so few and I cannot find any reason why this would happen for just these handful (so I assume it was a PDF reader error), I will just consult the results and fix these by hand.

In [97]:
dfResults.loc[917, ['Name2','Name3']] = ['Hannah Bach', 'Katherine Zenick']
dfResults.loc[598, ['Name2','Name3','Name4']] = ['Sean Faikish', 'Cason Wilburn', 'Thacher Scannell']
dfResults.loc[935, ['Name3','Name4']] = ['Lydia Hanlon', 'Kaitlynn Wheeler']

In [98]:
dfResults[(dfResults.Name3.isna()) & (~dfResults.Year3.isna())]

,Event,Category,Name1,Year1,Name2,Year2,Name3,Year3,Name4,Year4,School,QualifyingTime,Time,Place,Points,Division


In [19]:
dfResults[(dfResults.Name4.isna()) & (~dfResults.Year4.isna())]

,Event,Category,Name1,Year1,Name2,Year2,Name3,Year3,Name4,Year4,School,QualifyingTime,Time,Place,Points,Division


In [21]:
dfResults.describe(include='all')

,Event,Category,Name1,Year1,Name2,Year2,Name3,Year3,Name4,Year4,School,QualifyingTime,Time,Place,Points,Division
count,1949,1949,1949,1949,229,229,229,229,229,229,1949,1949,1949,1949,1949.000000,1949
unique,39,6,547,5,163,5,162,5,166,5,70,1662,1702,69,NaN,2
top,Event 3 Women 500 Yard Freestyle,Preliminaries,Katharine Berkoff,SR,Alex Walsh,SR,Callie Dickinson,SR,Abby Arens,SO,Florida,51.90,DFS,---,NaN,Women's DI
freq,84,1255,10,495,4,61,4,68,4,54,126,6,43,66,NaN,1035
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.658286,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.421645,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN


Looking at the counts, things look OK. For every entry, I expect every column to be filled except names/years 2-4 which are only used for relays. The previous issue meant that the counts for names/years weren't equal due to missing/NaN entries, but that issue is fixed by my by-hand updates.

I want to use datetime for the time columns so I can do time operations more easily, but there are some with minute values and some without. So I will convert all times with minutes first, then any that don't work will be converted as seconds. There are entries for DQs and DFSs that will be coerced into NaT.

The default is for the date to be Jan. 1, 1900, which I will leave as is. I'll only worry about differences in the time pieces anyway and I can use a column to differentiate meets rather than their dates anyway if I need to compare multiple meets (plus the meets are multi-day so it just becomes complicated).

In [99]:
dfResults.Time = pd.to_datetime(dfResults.Time, format='%M:%S.%f', errors='coerce').fillna(
                pd.to_datetime(dfResults.Time, format='%S.%f', errors='coerce'))
dfResults.QualifyingTime = pd.to_datetime(dfResults.QualifyingTime, format='%M:%S.%f', errors='coerce').fillna(
                pd.to_datetime(dfResults.QualifyingTime, format='%S.%f', errors='coerce'))
dfResults.describe(datetime_is_numeric=True)

,QualifyingTime,Time,Points
count,1949,1883,1949.000000
mean,1900-01-01 00:02:25.066541568,1900-01-01 00:02:27.072899328,3.658286
min,1900-01-01 00:00:17.930000,1900-01-01 00:00:18.250000,0.000000
25%,1900-01-01 00:00:51.249999872,1900-01-01 00:00:51.180000,0.000000
50%,1900-01-01 00:01:42.240000,1900-01-01 00:01:42.590000128,0.000000
75%,1900-01-01 00:02:10.020000,1900-01-01 00:02:12.160000,4.000000
max,1900-01-01 00:16:21.380000,1900-01-01 00:16:32.050000,40.000000
std,NaN,NaN,7.421645


In [38]:
dfResults.isna().sum()

Event                0
Category             0
Name1                0
Year1                0
Name2             1720
Year2             1720
Name3             1720
Year3             1720
Name4             1720
Year4             1720
School               0
QualifyingTime       0
Time                66
Place                0
Points               0
Division             0
dtype: int64

The qualifying and result times are now datetimes. The Time column has a few NaT entries due to DQs and DFSs, while the qualifying time does not have any since a qualifying time is required. There are also NaN's for the relay swimmer columns (name/year 2-4) for individual event entries.

I'll verify that the number of NaN's I see in the name/year columns is equal to the number of non-relay entries in the dataframe.

In [100]:
(~dfResults.Event.str.contains("Relay")).sum()

1720

Finally, in case I want to only consider finals or prelims separately, or want to drop the non-recorded times, I will split the dataframe into a few different subsets.

In [101]:
dfResultsFinals = dfResults[dfResults.Category.isin(
                            ['Timed Final Relay','Championship Final','Consolation Final', 'Timed Final Individual'])]
dfResultsPrelims = dfResults[dfResults.Category.isin(['Preliminaries','Swim-off'])]
dfResultsDropDQ = dfResults[~dfResults.Time.isnull()]

In [102]:
print(len(dfResults), len(dfResultsFinals), len(dfResultsPrelims))

1949 688 1261


In [103]:
dfResultsFinals.dtypes

Event                     object
Category                  object
Name1                     object
Year1                     object
Name2                     object
Year2                     object
Name3                     object
Year3                     object
Name4                     object
Year4                     object
School                    object
QualifyingTime    datetime64[ns]
Time              datetime64[ns]
Place                     object
Points                   float64
Division                  object
dtype: object

In [104]:
#dfResultsFinals[~dfResultsFinals.Time.isin(['DQ','DFS'])].Place.unique()
print(dfResultsFinals[~dfResultsFinals.Time.isnull()].Place.unique())
print(dfResultsFinals.Place.unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '19' '20' '21' '18' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41']
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '19' '20' '21' '---' '18' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41']


The Place column is mostly integers, although for DQ/DFS/DNS the Place given is '---'. For selections where I don't expect any of these and every entry should have a valid Place, then I could convert the column to integers. For now, I'll leave things as they are. For most Finals, there are Championship Finals and Consolation Finals, so 16 swimmers. But for Timed Finals (distance individual events and relays), there are more.

I've now cleaned the dataframes and split the data into a few different subsets that will be useful for different analyses. I can now tackle the questions I set forth to answer.

### Analyzing Data

#### Question 1: Which team(s) did best (and how should that be defined)?

The obvious answer to this question would be the team that scored the most points in the men's and women's competitions. And if I consider those 2 parts of the same team, the team with the most combined points would be the best. These answers can essentially be read right off the results (the PDF results add diving points which I will be ignoring for this analysis of purely swimming results).

But there are a few other considerations of which teams did the best at championships. Beyond just points scored, perhaps success should be measured in terms of the number of swimmers or total number of swims at championships or in a finals heat -- the most successful teams are the teams that sent the most swimmers to National Championships. Or, perhaps the opposite should be considered -- which teams had the most points per swimmer, regardless of the total number they sent.

Alternatively, maybe success should be defined by performance at this meet relative to other meets this season. Which swimmers and teams dropped the most time at this meet?

So, I will take these questions in turn and come up with a few notions of which teams had the "best" meet.

In [54]:
print(dfResults.School.unique())


['NC State' 'Arizona St' 'Florida' 'California' 'Indiana' 'Tennessee'
 'Louisville' 'Virginia' 'Auburn' 'Stanford' 'Alabama' 'Texas' 'Georgia'
 'Texas A&M' 'Wisconsin' 'Ohio St' 'Notre Dame' 'Michigan' 'Minnesota'
 'Pittsburgh' 'Harvard' 'Missouri' 'Virginia Tech' 'UNC' 'Kentucky'
 'Georgia Tech' 'Yale' 'Princeton' 'Florida St' 'Purdue' 'SMU'
 'West Virginia' 'Towson' 'Brown' 'Utah' 'Brigham Young'
 'Southern California' 'LSU' 'Penn St' 'Cal Baptist' 'Arizona' 'Air Force'
 'UNLV' 'Northwestern' 'Columbia' 'TCU' 'IUPUI' 'SIUC' 'South Carolina'
 'Arkansas' 'Duke' 'UCLA' 'Miami (Ohio)' 'Penn' 'Nebraska' 'Akron'
 'Oakland' 'GWU' 'Miami (FL)' 'San Diego St' 'Hawaii' 'UNC Asheville'
 'Buffalo' 'William & Mary' 'Nevada' 'Rice' 'Denver' "Florida Int'l"
 'Cincinnati' 'Washington St.']
70


In [108]:
print('Total unique schools:',dfResults.School.nunique())
print('Total women\'s teams:',dfResults[dfResults.Division=='Women\'s DI'].School.nunique())
print('Total men\'s teams:',dfResults[~(dfResults.Division=='Women\'s DI')].School.nunique())
print('Schools with both teams:',len(np.intersect1d(dfResults[dfResults.Division=='Women\'s DI'].School.unique(), 
                                                 dfResults[~(dfResults.Division=='Women\'s DI')].School.unique())))

Total unique schools: 70
Total women's teams: 55
Total men's teams: 49
Schools with both teams: 34


There are 55 unique schools in the Women's competition and 49 for the Men's (because of the 34 schools in both there are 70 total unique schools across both meets).

I'll make some leaderboards for teams scoring the most overall swimming points.

In [109]:
pointsTable = dfResults[['School','Points','Division']].groupby(['School','Division']).sum().reset_index()
pointsTable.groupby('School').sum().sort_values(by='Points',ascending=False).head(10)

,Points
School,
Texas,705.5
NC State,636.5
Virginia,619.5
California,616.0
Florida,527.0
Indiana,459.0
Stanford,454.5
Arizona St,449.0
Tennessee,398.5


In [110]:
pointsTable.sort_values(by='Points', ascending=False).head(10)

,School,Division,Points
95,Virginia,Women's DI,541.5
15,California,Men's DI,479.0
6,Arizona St,Men's DI,430.0
50,NC State,Men's DI,373.5
83,Texas,Women's DI,365.5
21,Florida,Men's DI,353.0
82,Texas,Men's DI,340.0
78,Stanford,Women's DI,333.0
34,Indiana,Men's DI,275.0
41,Louisville,Women's DI,266.0


In [111]:
pointsTable[pointsTable.Division=='Women\'s DI'].sort_values(by='Points', ascending=False).head(10)

,School,Division,Points
95,Virginia,Women's DI,541.5
83,Texas,Women's DI,365.5
78,Stanford,Women's DI,333.0
41,Louisville,Women's DI,266.0
51,NC State,Women's DI,263.0
60,Ohio St,Women's DI,216.0
81,Tennessee,Women's DI,214.0
35,Indiana,Women's DI,184.0
22,Florida,Women's DI,174.0
16,California,Women's DI,137.0


In [112]:
pointsTable[~(pointsTable.Division=='Women\'s DI')].sort_values(by='Points', ascending=False).head(10)

,School,Division,Points
15,California,Men's DI,479.0
6,Arizona St,Men's DI,430.0
50,NC State,Men's DI,373.5
21,Florida,Men's DI,353.0
82,Texas,Men's DI,340.0
34,Indiana,Men's DI,275.0
80,Tennessee,Men's DI,184.5
96,Virginia Tech,Men's DI,127.0
77,Stanford,Men's DI,121.5
9,Auburn,Men's DI,121.0


#### Interpretation:
From the above, we can see the top schools by the number of points scored for a few selections. For the Women's meet the top schools were Virginia, Texas, Stanford. For the Men's the top 3 were Cal, Arizona St., and NC State. If we compare the top performing teams in the two groups, the highest scorers were the Virginia Women, Cal Men, and Arizona St. Men. Finally, when combining men's and women's scores, the top 3 schools were Texas, NC State, and Virginia.

Next, we look at the related but separate question of which schools led in terms of number of swimmers.

First, I'll only consider swimmers in individual events.

In [123]:
swimmersTable = dfResults[~dfResults.Event.str.contains("Relay")][['School','Name1','Points','Division']].groupby(
    ['School','Division']).agg({'Name1':'nunique','Points':'sum'}).reset_index()

In [124]:
swimmersTable.sort_values(by='Name1', ascending=False).head(10)

,School,Division,Name1,Points
94,Virginia,Women's DI,17,341.5
50,NC State,Men's DI,17,215.5
22,Florida,Women's DI,16,78.0
21,Florida,Men's DI,16,173.0
15,California,Men's DI,16,321.0
78,Stanford,Women's DI,15,183.0
60,Ohio St,Women's DI,15,94.0
82,Texas,Men's DI,15,216.0
6,Arizona St,Men's DI,15,270.0
83,Texas,Women's DI,14,219.5


Consistency check to make sure I didn't mess anything up while grouping and summing: do these values match the points totals for teams from individual events?

In [125]:
dfResults[~dfResults.Event.str.contains("Relay")][['School','Points','Division']].groupby(
    ['School','Division']).sum().reset_index().sort_values(by='Points', ascending=False)

,School,Division,Points
94,Virginia,Women's DI,341.5
15,California,Men's DI,321.0
6,Arizona St,Men's DI,270.0
83,Texas,Women's DI,219.5
82,Texas,Men's DI,216.0
...,...,...,...
79,TCU,Men's DI,0.0
49,Missouri,Women's DI,0.0
33,IUPUI,Men's DI,0.0
31,Harvard,Men's DI,0.0


Matching teams in the two previous tables, I can see that they give the same results.
Now, I'll add a points per swimmer column to the table.

In [126]:
swimmersTable['PointsPerSwimmer'] = swimmersTable.Points / swimmersTable.Name1
swimmersTable.rename(columns = {'Name1':'Swimmers'}, inplace=True)
swimmersTable.sort_values(by='PointsPerSwimmer',ascending=False).head(10)

,School,Division,Swimmers,Points,PointsPerSwimmer
38,LSU,Men's DI,1,43.5,43.500000
94,Virginia,Women's DI,17,341.5,20.088235
15,California,Men's DI,16,321.0,20.062500
6,Arizona St,Men's DI,15,270.0,18.000000
56,Notre Dame,Men's DI,3,52.0,17.333333
83,Texas,Women's DI,14,219.5,15.678571
34,Indiana,Men's DI,9,139.0,15.444444
82,Texas,Men's DI,15,216.0,14.400000
39,LSU,Women's DI,4,53.0,13.250000
81,Tennessee,Women's DI,10,132.0,13.200000


#### Interpretation

When looking at the number of swimmers and points per swimmer we see lots of the same names the points table. Virginia Women did well in points, swimmmers, and points per swimmer, as did Cal and Arizona St. Men.

One standout when looking at points per swimmer is LSU Men who took 43.5 points with their only swimmer. I'll take a closer look at these results.

In [127]:
dfResults[(dfResults.School=='LSU') & (dfResults.Division=='Men\'s DI')]

,Event,Category,Name1,Year1,Name2,Year2,Name3,Year3,Name4,Year4,School,QualifyingTime,Time,Place,Points,Division
188,Event 5 Men 50 Yard Freestyle,Championship Final,Brooks Curry,SR,NaN,NaN,NaN,NaN,NaN,NaN,LSU,1900-01-01 00:00:18.720,1900-01-01 00:00:18.760,4,15.0,Men's DI
204,Event 5 Men 50 Yard Freestyle,Preliminaries,Brooks Curry,SR,NaN,NaN,NaN,NaN,NaN,NaN,LSU,1900-01-01 00:00:18.940,1900-01-01 00:00:18.720,4,0.0,Men's DI
397,Event 10 Men 200 Yard Freestyle,Championship Final,Brooks Curry,SR,NaN,NaN,NaN,NaN,NaN,NaN,LSU,1900-01-01 00:01:31.940,1900-01-01 00:01:31.300,4,15.0,Men's DI
413,Event 10 Men 200 Yard Freestyle,Preliminaries,Brooks Curry,SR,NaN,NaN,NaN,NaN,NaN,NaN,LSU,1900-01-01 00:01:33.150,1900-01-01 00:01:31.940,4,0.0,Men's DI
693,Event 17 Men 100 Yard Freestyle,Championship Final,Brooks Curry,SR,NaN,NaN,NaN,NaN,NaN,NaN,LSU,1900-01-01 00:00:41.170,1900-01-01 00:00:41.030,5,13.5,Men's DI
708,Event 17 Men 100 Yard Freestyle,Preliminaries,Brooks Curry,SR,NaN,NaN,NaN,NaN,NaN,NaN,LSU,1900-01-01 00:00:41.860,1900-01-01 00:00:41.170,4,0.0,Men's DI


All of LSU Men's swims are from Brooks Curry, who totaled 43.5 points across 3 events. Other teams that picked up lots of points per swimmer but didn't have enough total swimmers to get them into the top of the team points totals include Notre Dame's Men (52 points from 3 swimmers), Indiana's Men (139 points from 9 swimmers), and LSU's Women (53 points from 4 swimmers). This also highlights Brooks's great result: compared to the LSU women, he achieved 82% of their score from 4 swimmers by himself.

I will also take a look at teams on the other end of the previous table: those who did not score many points.

In [135]:
swimmersTable.sort_values(by=['Points','Swimmers'], ascending=[True,False]).head(10)

,School,Division,Swimmers,Points,PointsPerSwimmer
8,Arkansas,Women's DI,6,0.0,0.0
74,South Carolina,Women's DI,6,0.0,0.0
24,Florida St,Men's DI,3,0.0,0.0
49,Missouri,Women's DI,3,0.0,0.0
72,San Diego St,Women's DI,3,0.0,0.0
5,Arizona,Women's DI,2,0.0,0.0
54,Northwestern,Men's DI,2,0.0,0.0
57,Notre Dame,Women's DI,2,0.0,0.0
64,Pittsburgh,Women's DI,2,0.0,0.0
75,Southern California,Men's DI,2,0.0,0.0


The Arkansas and South Carolina Women's team managed to qualify 6 swimmers for national championships but did not score any points.

In [136]:
len(swimmersTable[swimmersTable.Points==0])

36

Overall, 36 teams ended the meet with 0 points from individual events.

In [137]:
dfTmp = swimmersTable[['School','Points']].groupby('School').sum().reset_index()
len(dfTmp[dfTmp.Points==0])

26

Combining Men's and Women's teams, 26 schools left the championships with no points.

I will also consider another metric: points per number of swims (rather than swimmers) or average place. This picks up teams that tended to have their swimmers score lots of points (high points finishes). I expect to find lots of the same teams, but there's also a possibility of finding something similar to the LSU/Brooks Curry finding. Perhaps there are teams that didn't necessarily get lots of swimmers into finals and score lots of points, but when they did they tended to place well.

First I will consider total event entries per school. All entries will have a prelim entry, and those that made finals will have a repeat (except for the 1650 which has just one swim, the 'timed final'). I am only interested in counting each case as one "swimmer entry." So I want a table of total number of prelim swims per team, along with total points scored. I will also consider which schools did best at converting prelims entries into finals qualifications.

Then, I will limit the scope to only finals swim and which schools converted finals swims into the most points and the highest average finishing places.

In [138]:
pointsTable = dfResults[~dfResults.Event.str.contains("Relay")][
    ['School','Points','Name1','Division','Category']]
pointsTable.Category.unique()

array(['Championship Final', 'Consolation Final', 'Preliminaries',
       'Timed Final Individual', 'Swim-off'], dtype=object)

"Timed Final Individual" is both a prelim and final, so I will have to count it as both. I'll eventually count every swim in this type of event as if it were in both Preliminaries and the Championship Final. This messes with the analysis a little because there will be many more than just 8 swimmers in the championship final for this event, but everyone has a chance to win the championship final points, so I think this is the best option. A swim-off is essentially a re-do of a prelim so it shouldn't really be counted here.

I'll make new columns that are bools for each category that I can easily sum over.

In [139]:
pointsTable['Prelim'] = (pointsTable.Category == 'Preliminaries') | (pointsTable.Category == 'Timed Final Individual')
pointsTable['BFinal'] = pointsTable.Category == 'Consolation Final'
pointsTable['AFinal'] = pointsTable.Category == 'Championship Final'
pointsTable['TimedFinal'] = pointsTable.Category == 'Timed Final Individual'
pointsTable['BPoints'] = pointsTable[pointsTable.Category == 'Consolation Final'].Points
pointsTable['APoints'] = pointsTable[pointsTable.Category == 'Championship Final'].Points
pointsTable['TimedPoints'] = pointsTable[pointsTable.Category == 'Timed Final Individual'].Points
pointsTable.head()

,School,Points,Name1,Division,Category,Prelim,BFinal,AFinal,TimedFinal,BPoints,APoints,TimedPoints
44,Texas,20.0,Luke Hobson,Men's DI,Championship Final,False,False,True,False,NaN,20.0,NaN
45,Texas,17.0,David Johnston,Men's DI,Championship Final,False,False,True,False,NaN,17.0,NaN
46,Georgia,16.0,Jake Magahey,Men's DI,Championship Final,False,False,True,False,NaN,16.0,NaN
47,Wisconsin,15.0,Jake Newmark,Men's DI,Championship Final,False,False,True,False,NaN,15.0,NaN
48,Florida,14.0,Jake Mitchell,Men's DI,Championship Final,False,False,True,False,NaN,14.0,NaN


In [143]:
pointsTable = pointsTable.groupby(['School','Division']).sum().reset_index()
pointsTable['pointsPerPrelim'] = pointsTable.Points / (pointsTable.Prelim + pointsTable.TimedFinal)
pointsTable['pointsPerFinal'] = pointsTable.Points / (pointsTable.BFinal + pointsTable.AFinal + pointsTable.TimedFinal)
pointsTable['BPointsPercentage'] = pointsTable.BPoints / pointsTable.Points
pointsTable['APointsPercentage'] = pointsTable.APoints / pointsTable.Points
pointsTable['TimedPointsPercentage'] = pointsTable.TimedPoints / pointsTable.Points
pointsTable.head()

,School,Division,Points,Prelim,BFinal,AFinal,TimedFinal,BPoints,APoints,TimedPoints,pointsPerPrelim,pointsPerFinal,BPointsPercentage,APointsPercentage,TimedPointsPercentage
0,Air Force,Men's DI,2.0,3,1,0,0,2.0,0.0,0.0,0.666667,2.000000,1.000000,0.000000,0.000000
1,Akron,Women's DI,5.0,9,2,0,0,5.0,0.0,0.0,0.555556,2.500000,1.000000,0.000000,0.000000
2,Alabama,Men's DI,21.0,13,2,1,1,8.0,11.0,2.0,1.500000,5.250000,0.380952,0.523810,0.095238
3,Alabama,Women's DI,73.0,20,4,2,1,22.0,31.0,20.0,3.476190,10.428571,0.301370,0.424658,0.273973
4,Arizona,Men's DI,0.0,3,0,0,0,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,NaN


Now I have a table that has a row for each team with the total points scored and number of swims in each category of prelim/final as well as how many points each team got from each type of final and the percentage of points from each type of final.

Now, I'll look at the top 10 teams by points per (prelim) swim.

In [144]:
pointsTable[['School','Division','Points','Prelim','pointsPerPrelim']].sort_values(by='pointsPerPrelim', ascending=False).head(10)

,School,Division,Points,Prelim,pointsPerPrelim
38,LSU,Men's DI,43.5,3,14.500000
15,California,Men's DI,321.0,43,7.295455
94,Virginia,Women's DI,341.5,48,6.830000
69,SIUC,Men's DI,13.0,2,6.500000
83,Texas,Women's DI,219.5,32,6.271429
34,Indiana,Men's DI,139.0,23,6.043478
56,Notre Dame,Men's DI,52.0,8,5.777778
6,Arizona St,Men's DI,270.0,45,5.625000
82,Texas,Men's DI,216.0,39,5.142857
35,Indiana,Women's DI,102.0,19,4.857143


#### Interpretation

At the top of the points per prelim metric we see the LSU Men's team again, who scored 43.5 points off of 3 Brooks Curry entries for 14.5 points per prelim. This is nearly twice as much as the next nearest team. Cal's Men's team managed to score an impressive 7.3 points per prelim with 43 entries. The Virginia Women round out the top 3 with 6.8 points per prelim and 48 total prelims. Of note in this top 10 are 2 smaller teams who managed to perform well: SIUC's Men's Team (2 entries, 6.5 points per prelim) and Notre Dame's Men's Team (8 entries, 5.8 points per prelim).

In [145]:
dfResults[(dfResults.School=='SIUC') & (dfResults.Division=='Men\'s DI')]

,Event,Category,Name1,Year1,Name2,Year2,Name3,Year3,Name4,Year4,School,QualifyingTime,Time,Place,Points,Division
526,Event 12 Men 100 Yard Backstroke,Consolation Final,Ruard Van Renen,FR,NaN,NaN,NaN,NaN,NaN,NaN,SIUC,1900-01-01 00:00:45.170,1900-01-01 00:00:44.670,9,9.0,Men's DI
543,Event 12 Men 100 Yard Backstroke,Preliminaries,Ruard Van Renen,FR,NaN,NaN,NaN,NaN,NaN,NaN,SIUC,1900-01-01 00:00:44.890,1900-01-01 00:00:45.170,10,0.0,Men's DI
645,Event 16 Men 200 Yard Backstroke,Consolation Final,Ruard Van Renen,FR,NaN,NaN,NaN,NaN,NaN,NaN,SIUC,1900-01-01 00:01:39.730,1900-01-01 00:01:40.220,13,4.0,Men's DI
658,Event 16 Men 200 Yard Backstroke,Preliminaries,Ruard Van Renen,FR,NaN,NaN,NaN,NaN,NaN,NaN,SIUC,1900-01-01 00:01:40.640,1900-01-01 00:01:39.730,10,0.0,Men's DI


Ruard Van Renen from SIUC was the only SIUC swimmer at the meet, making 2 Consolation Finals, scoring a total of 13 points by himself. (This strong peformance also helps highlight once again how impressive Brooks Curry's 14.5 points per event as the only LSU Men's entry is.)

Now I'll consider teams that were able to turn prelim swims into finals entries.

In [146]:
pointsTable.assign(FinalPercentage = 
                   (pointsTable.AFinal + pointsTable.BFinal + pointsTable.TimedFinal)/pointsTable.Prelim,
                  Final = pointsTable.AFinal + pointsTable.BFinal + pointsTable.TimedFinal)[
    ['School','Division','Points','Prelim','Final','FinalPercentage']].sort_values(
    by='FinalPercentage',ascending=False).drop('FinalPercentage',axis=1).reset_index(drop=True).head(10)

,School,Division,Points,Prelim,Final
0,SIUC,Men's DI,13.0,2,2
1,LSU,Men's DI,43.5,3,3
2,Notre Dame,Men's DI,52.0,8,7
3,Indiana,Women's DI,102.0,19,12
4,California,Men's DI,321.0,43,26
5,Arizona St,Men's DI,270.0,45,27
6,Virginia,Women's DI,341.5,48,28
7,Texas,Women's DI,219.5,32,18
8,Louisville,Women's DI,140.0,28,15
9,NC State,Women's DI,155.0,34,18


#### Interpretation

We see lots of repeat top performing teams here, many teams in the top 10 have been discussed as they have been in other top 10s. The top 10 teams below Indiana's Women at #4 (12 of 17 entries in finals) are all large teams that had many swimmer in prelims and finals and took home large points hauls. As discussed, SIUC and LSU's swimmers made finals in all their events. Notre Dame's Men's Team converted 7 of 8 swims into finals (6 consolation finals + 1 timed 1650 final). The lone event they missed the finals of was the 50 yard freestyle, where Chris Guiliano finished 23rd in the prelims. In the 100 Backstroke prelims, Tommy Janton finished tied for 16th so had to go to a swim-off, which he won to earn the last spot in the consolation final.

We can also consider which teams failed to make finals, despite many prelims entries.

In [147]:
dfTmp = pointsTable.assign(FinalPercentage = 
                   (pointsTable.AFinal + pointsTable.BFinal + pointsTable.TimedFinal)/pointsTable.Prelim,
                  Final = pointsTable.AFinal + pointsTable.BFinal + pointsTable.TimedFinal)[
    ['School','Division','Points','Prelim','Final','FinalPercentage']].sort_values(
    by=['FinalPercentage','Prelim'],ascending=[True,False]).drop('FinalPercentage',axis=1).reset_index(drop=True)
dfTmp[dfTmp.Final==0].head(10)

,School,Division,Points,Prelim,Final
0,South Carolina,Women's DI,0.0,15,0
1,Arkansas,Women's DI,0.0,14,0
2,Missouri,Women's DI,0.0,7,0
3,San Diego St,Women's DI,0.0,7,0
4,Arizona,Women's DI,0.0,6,0
5,Notre Dame,Women's DI,0.0,5,0
6,Pittsburgh,Women's DI,0.0,5,0
7,Southern California,Men's DI,0.0,4,0
8,Arizona,Men's DI,0.0,3,0
9,Brigham Young,Men's DI,0.0,3,0


Despite qualifying a lot of swimmers for national championships, South Carolina Women and Arkansas Women were unable to convert any of their 15 and 14 prelim swims, respectively, into finals appearances. These 2 programs had twice as many as any other teams that secured 0 inividual finals.

We can repeat the previous steps, only considering the Championship (A) Final.

In [148]:
pointsTable.assign(AFinalPercentage = pointsTable.AFinal / pointsTable.Prelim)[
    ['School','Division','Points','Prelim','AFinal','AFinalPercentage']].sort_values(
    by='AFinalPercentage',ascending=False).drop('AFinalPercentage',axis=1).head(10)

,School,Division,Points,Prelim,AFinal
38,LSU,Men's DI,43.5,3,3
15,California,Men's DI,321.0,43,18
94,Virginia,Women's DI,341.5,48,19
83,Texas,Women's DI,219.5,32,12
32,Hawaii,Women's DI,11.5,3,1
47,Minnesota,Women's DI,13.0,3,1
82,Texas,Men's DI,216.0,39,12
34,Indiana,Men's DI,139.0,23,7
6,Arizona St,Men's DI,270.0,45,13
41,Louisville,Women's DI,140.0,28,8


#### Interpretation

Among familiar names and impressive results from larger teams, there are 2 teams that got 1 Championship Final entry from just 3 prelim entries: Hawaii Women (Laticia-Leigh Tansom, 7th in 100 free) and Minnesota Women (Megan Van Berkom, 6th in 400 IM).

Now, rather than points per swim, I'll look at average place finish among finals swims.

In [150]:
ATable = dfResults[~(dfResults.Event.str.contains("Relay")) & 
                   (dfResults.Category.isin(['Championship Final','Consolation Final','Timed Final Individual']))][
    ['School','Name1','Points','Division','Category','Place']]
ATable.Place = ATable.Place.astype('int')
ATable.groupby(['School','Division']).agg({
    'Name1': 'count', 'Points':'sum', 'Place':'mean'
}).sort_values(by=['Place','Name1'], ascending=[True,False]).reset_index().head(10)

,School,Division,Name1,Points,Place
0,LSU,Women's DI,3,53.0,2.000000
1,LSU,Men's DI,3,43.5,4.333333
2,Utah,Men's DI,1,14.0,5.000000
3,Indiana,Men's DI,11,139.0,6.000000
4,Minnesota,Women's DI,1,13.0,6.000000
5,California,Men's DI,26,321.0,6.192308
6,Georgia,Men's DI,6,70.0,6.666667
7,Hawaii,Women's DI,1,11.5,7.000000
8,Virginia Tech,Men's DI,5,59.0,7.200000
9,Virginia,Women's DI,28,341.5,7.285714


#### Interpretation

The above table gives teams with the lowest average place, among entries in Final heats. The LSU Women and Men both had 3 finals swims, with an average finsh of 2nd place for the Women and 4.33 for the Men. Utah Men had 1 finals swim and placed 5th. We can see that in this ranking, which rewards the highest averages, the top 10 is mostly filled with teams that only had a few finals swims. Among the top 10, the teams that had both a large number of finals swims and a high average finish were Indiana Men (11 swims, 6.0 average), Cal Men (26 swims, 6.2 average), and Virginia Women (28 swims, 7.3 average). The teams that scored the most swimming points (Cal Men and Virginia Women) got lots of swimmers into finals and they finished highly on average.

In [151]:
# Look at average points per A final, rather than sum of points in previous table
ATable.groupby(['School','Division']).agg({
    'Name1': 'count', 'Points':'mean', 'Place':'mean'
}).sort_values(by=['Points','Name1'], ascending=[False,False]).reset_index().head(10)

,School,Division,Name1,Points,Place
0,LSU,Women's DI,3,17.666667,2.000000
1,LSU,Men's DI,3,14.500000,4.333333
2,Utah,Men's DI,1,14.000000,5.000000
3,Minnesota,Women's DI,1,13.000000,6.000000
4,Indiana,Men's DI,11,12.636364,6.000000
5,California,Men's DI,26,12.346154,6.192308
6,Virginia,Women's DI,28,12.196429,7.285714
7,Texas,Women's DI,18,12.194444,7.666667
8,Virginia Tech,Men's DI,5,11.800000,7.200000
9,Wisconsin,Women's DI,7,11.714286,8.571429


#### Interpretation

This table shows how the average finish converts into average points per A Final swim. The top 10 is mostly the same when looking at average points or average finishing place; however, because points are not perfectly linear with place (the gap in points awarded to finishing places is different for different places from 1-16) they are not exactly the same. It matters exactly how you end up with a certain average place. For example, Minnesota's Women and Indiana's Men both had an average finish of 6th. However, Minnesota had more points on average per swim (just 1 swim for Minnesota compared to 11 for Indiana). Across the 11 swims, Indiana on average placed 6th but because of the points allocations, the smaller points awarded for finishes below 6th couldn't balance out the larger rewards for top 6 finishes, so the average points per swim was smaller than the points awarded for a single 6th place.

I'll look at some individual swimmer accomplishments as well. First, I'll add a column with a value equal to the number of total unique swimmers on each team with at least one finals swim.

In [186]:
ATable['TeamSize'] = ATable.groupby(['School','Division']).Name1.transform('nunique')
ATable.groupby(['Name1','School']).agg({
    'Place':'count','Points':'sum','TeamSize':'mean'
}).sort_values(by=['Points','TeamSize'], ascending=[False,True]).reset_index().head(20)

,Name1,School,Place,Points,TeamSize
0,Kate Douglass,Virginia,3,60.0,13.0
1,Leon Marchand,Arizona St,3,60.0,13.0
2,Gretchen Walsh,Virginia,3,57.0,13.0
3,Josh Liendo,Florida,3,54.0,14.0
4,Maggie MacNeil,LSU,3,53.0,1.0
5,Destin Lasco,California,3,53.0,12.0
6,Alex Walsh,Virginia,3,53.0,13.0
7,Claire Curzan,Stanford,3,51.0,12.0
8,Hugo Gonzalez,California,3,50.0,12.0
9,Torri Huske,Stanford,3,50.0,12.0


#### Interpretation

Here is a table of the top 20 individual performers, sorted by points scored. The Place column is the number of individual finals swims they had, and the TeamSize column is the number of total swimmers on their team with finals swims. Both Kate Douglass and Leon Marchand had 60 points from 3 individual wins (20 points for first place). The top 10 individual swimmers are mostly from large programs, except for Maggie MacNeil from LSU who was the only member of the LSU Women's team to make finals. She scored 53 points, more than Brooks Curry who was the only LSU Men's team member. In places 11-20, the swimmers tended to be from slightly smaller teams than the top 10.

### Final comments

Exploring the 2023 NCAA Championship Meet results highlighted a few things for me. The Cal Men and Virginia Women did not score the most points by qualifying many swimmers and accumulating lots of points because of having more entries. They had an impressive ability to get their qualifiers through to finals and placed well in the finals.

Some of the smaller teams that had standout performances that I found were SIUC Men, LSU Men and Women, Hawaii Women, Minnesota Women, Utah Men. These teams did not have the overall size to be able to compete with the larger programs at this championship meet, but they had impressive performances nonetheless. They managed to get finals swims and score well with the few swimmers they had. From this group, the performers that were quite impressive to me were Brooks Curry and Maggie MacNeil from LSU, who were the only members of the Men's and Women's teams respectively to get into finals (Brooks was the only Men's qualifier at all, Maggie had teammates there who didn't qualify for finals), but they managed to come away with impressive individual points hauls.

### To-do

- Are there any differences in performance based on a swimmer's year?
- What are the average improvements from qualifying times to preliminaries to finals? Do any factors impact this? Are certain schools better/worse at dropping time in one or both of these steps?

#### Longer term

- Get results from many years to be able to make comparisons across years